In [2]:
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
import os

In [3]:
path_to_file = os.path.join(".", "Resources", "Rowan County Adoptable Dog List - Cat Info.csv")
RCA_cat_df = pd.read_csv(path_to_file)
RCA_cat_df.rename(columns = {"Description":"Description/Breed"}, inplace = True)
cleanRCA_cat_df = RCA_cat_df[["Animal ID", "Description/Breed", "Sex", "Intake"]].copy()
cleanRCA_cat_df["Shelter County"]= "Rowan"
cleanRCA_cat_df.head()

,Animal ID,Description/Breed,Sex,Intake,Shelter County
0,A124749,DSH Brw Tabby W/Wht,Female,Stray,Rowan
1,A124782,DSH Brn Tab,Male,Stray,Rowan
2,A124797,DSH Silver Tab,Male,Stray,Rowan
3,A124800,DSH Dilute Torti,Female,Stray,Rowan
4,A124661,DSH Blk/Wht,Male,Stray,Rowan


In [4]:
path_to_file = os.path.join(".", "Resources", "Rowan County Adoptable Dog List - Dog Info.csv")
RCA_dog_df = pd.read_csv(path_to_file)
RCA_dog_df.rename(columns = {"Breed":"Description/Breed"}, inplace = True)
cleanRCA_dog_df = RCA_dog_df[["Animal ID", "Description/Breed", "Sex", "Intake"]].copy()
cleanRCA_dog_df["Shelter County"]= "Rowan"
cleanRCA_dog_df.head()

,Animal ID,Description/Breed,Sex,Intake,Shelter County
0,A122246,Catahoula Mix,Male,Owner,Rowan
1,A125012,Chih Mix,Female,Stray,Rowan
2,A125014,Lab Mix,Male,Stray,Rowan
3,A125017,Shepherd Mix,Female,Stray,Rowan
4,A125031,German Shep.,Male,Owner,Rowan


In [5]:
path_to_file = os.path.join(".", "Resources", "SoCo-Animal_Shelter_Intake_and_Outcome.csv")
SoCO_animals_df = pd.read_csv(path_to_file)
SoCO_animals_df.rename(columns = {"Breed":"Description/Breed", "Intake Subtype":"Intake"}, inplace = True)
cleanSoCo_animals_df = SoCO_animals_df[["Animal ID", "Description/Breed", "Sex", "Intake"]].copy()
cleanSoCo_animals_df["Shelter County"]= "SoCo"
cleanSoCo_animals_df.head()

,Animal ID,Description/Breed,Sex,Intake,Shelter County
0,A395686,GERM SHEPHERD,Female,FIELD,SoCo
1,A379968,BEAGLE,Neutered,FIELD,SoCo
2,A315553,LABRADOR RETR/MIX,Spayed,OVER THE COUNTER,SoCo
3,A398868,GERM SHEPHERD/MIX,Unknown,FLD_CRUEL,SoCo
4,A374683,DOMESTIC SH,Neutered,OVER THE COUNTER,SoCo


In [6]:
AnimalStuff = pd.concat([cleanSoCo_animals_df,cleanRCA_dog_df,cleanRCA_cat_df], axis = 0)

In [7]:
total = 0

for df in [cleanSoCo_animals_df,cleanRCA_dog_df,cleanRCA_cat_df]:
    total += df.shape[0]
    
print(total)    

20245


In [26]:
AnimalStuff_Groupby = AnimalStuff.groupby('Animal ID')['Description/Breed','Sex','Intake', 'Shelter County'].first()
print(AnimalStuff_Groupby)


<ipython-input-26-2dcb0c37e298>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  AnimalStuff_Groupby = AnimalStuff.groupby('Animal ID')['Description/Breed','Sex','Intake', 'Shelter County'].first()


           Description/Breed       Sex    Intake Shelter County
Animal ID                                                      
A005178    LABRADOR RETR/MIX  Neutered     FIELD           SoCo
A006149        GERM SHEPHERD    Spayed  VET_HOSP           SoCo
A007213          DOMESTIC SH    Spayed     FIELD           SoCo
A007746      MAREMMA SHEEPDG    Spayed     FIELD           SoCo
A007771      ENG SPRNGR SPAN      Male  VET_HOSP           SoCo
...                      ...       ...       ...            ...
A401043               BENGAL    Female     PHONE           SoCo
A401053       SIBERIAN HUSKY      Male     FIELD           SoCo
A401058        IRISH TERRIER      Male     PHONE           SoCo
A401059        IRISH TERRIER      Male     PHONE           SoCo
A401065         PIT BULL/MIX    Female     FIELD           SoCo

[18356 rows x 4 columns]


In [27]:
pg_user = 'postgres'
pg_password = 'postgres'
db_name = 'Shelter_DB'

connection_string = f"{pg_user}:{pg_password}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')

AnimalStuff_Groupby.to_csv("Animanls.csv", index = "id")

In [28]:
engine.table_names()

['shelterdata']

In [29]:
cleanSoCo_animals_df.to_sql(name='shelterdata', con=engine, if_exists='append', index=False)